In [3]:
def process_hourly(df):
    df['taxi_total'] = df['taxi_in'] + df['taxi_out']
    arr_hourly = df.groupby('arr_hour', as_index=False).agg(arr_hourly_arr=('arr_delay', 'mean'), arr_hourly_dep=('dep_delay', 'mean'), arr_hour_taxi=('taxi_total', 'mean'), late=('late_binary', 'sum'), ltot=('late_binary', 'size'), reduce=('reduced_delay_flight', 'sum'), rtot=('reduced_delay_flight', 'size'))
    arr_hourly['arr_hourly_percent_late'] = arr_hourly['late'] / arr_hourly['ltot'] * 100
    arr_hourly['arr_hourly_percent_reduced_delay'] = arr_hourly['reduce'] / arr_hourly['rtot'] * 100
    arr_hourly = arr_hourly.drop(columns=['late', 'ltot', 'reduce', 'rtot'])
    
    dep_hourly = df.groupby('dep_hour', as_index=False).agg(dep_hourly_arr=('arr_delay', 'mean'), dep_hourly_dep=('dep_delay', 'mean'), dep_hourly_taxi=('taxi_total', 'mean'), late=('late_binary', 'sum'), tot=('late_binary', 'size'), red=('reduced_delay_flight', 'sum'), rtot=('reduced_delay_flight', 'size'))
    dep_hourly['dep_hourly_percent_late'] = dep_hourly['late'] / dep_hourly['tot'] * 100
    dep_hourly['dep_hourly_percent_reduced'] = dep_hourly['red'] / dep_hourly['rtot'] * 100
    dep_hourly = dep_hourly.drop(columns=['late', 'red', 'tot', 'rtot'])
    
    df.merge(arr_hourly, on='arr_hour')
    df.merge(dep_hourly, on='dep_hour')
    return df

In [4]:
def process_weekly(df):
    df['taxi_total'] = df['taxi_in'] + df['taxi_out']
    weekday = df.groupby('weekday', as_index=False).agg(weekday_arr=('arr_delay', 'mean'), weekday_dep=('dep_delay', 'mean'), weekday_taxi=('taxi_total', 'mean'), late=('late_binary', 'sum'), tot=('late_binary', 'size'), red=('reduced_delay_flight', 'sum'), rtot=('reduced_delay_flight', 'size'))
    weekday['weekly_percent_late'] = weekday['late'] / weekday['tot'] * 100
    weekday['weekly_reduced_delay'] = weekday['red'] / weekday['rtot'] * 100
    weekday = weekday.drop(columns=['red', 'late', 'rtot', 'tot'])

    df = df.merge(weekday, on='weekday')
    return df

In [5]:
def process_carriers(df):
    """
    returns new columns for avg taxi in and out based on airports, and total taxi on tail num
    """
    carriers = df.groupby('op_unique_carrier', as_index=False).agg(carrier_arr=('arr_delay', 'mean'), carrier_dep=('dep_delay', 'mean'), carrier_taxiin=('taxi_in', 'mean'), carrier_taxiout=('taxi_out', 'mean'), carrier_late=('late_binary', 'sum'), carrier_total=('late_binary', 'size'), carrier_reduce_delay=('reduced_delay_flight', 'sum'), reduced_total=('reduced_delay_flight', 'size'))
    carriers['carrier_percent_late'] = carriers['carrier_late'] / carriers['carrier_total'] * 100
    carriers['carrier_taxi_total'] = carriers['carrier_taxiin'] + carriers['carrier_taxiout']
    carriers['percent_carrier_delay_reduced'] = carriers['carrier_reduce_delay'] / carriers['reduced_total'] * 100
    carriers = carriers.drop(columns=['carrier_reduce_delay', 'reduced_total', 'carrier_taxiout', 'carrier_taxiin', 'carrier_total', 'carrier_late'])
    df = df.merge(carriers, on='op_unique_carrier')
    return df